# Fixed Income Final Project

In [2]:
import pandas as pd
import numpy as np
import os
import scipy.optimize as optimize
from sklearn import linear_model
import math

#NOTEBOOK_ROOT = "~/Desktop/Fixed Income Project"
#UK_PATH = os.path.join( NOTEBOOK_ROOT, "UK_data")

Day1_data = pd.read_csv("FilePath.csv")

## Data Preparation

In [2]:
# Experiment one day data
data = Day1_data

In [3]:
data.head()

,Tenor,CUSIP,Description,Price,Yield,Source,Update,Expiry Date,Months to Expiry,Coupon
0,1M,AT256668@BGN Corp,UKTB 0 12/31/18 Corp,99.9480,0.7033,BGN,2018/12/3,2018/12/31,0.933333,0.00
1,3M,AU235812@BVAL Corp,UKTB 0 03/04/19 Corp,99.8372,0.6612,BVAL,2018/12/3,2019/3/4,3.033333,0.00
2,1Y,EJ933497@BGN Corp,UKT 1 ¾ 07/22/19 Corp,100.6390,0.7361,BGN,2018/12/3,2019/7/22,7.700000,1.75
3,2Y,EK459367@BGN Corp,UKT 2 07/22/20 Corp,102.0735,0.7204,BGN,2018/12/3,2020/7/22,19.900000,2.00
4,3Y,UV650560@BGN Corp,UKT 1 ½ 01/22/21 Corp,101.6365,0.7254,BGN,2018/12/3,2021/1/22,26.033333,1.50


In [4]:
act_price = data["Price"]

coupon = data["Coupon"]

In [5]:
print(coupon)

0     0.000
1     0.000
2     1.750
3     2.000
4     1.500
5     0.500
6     0.750
7     2.750
8     2.000
9     1.500
10    1.250
11    1.625
12    4.750
13    4.250
14    1.750
15    4.500
16    1.500
17    1.750
18    3.500
Name: Coupon, dtype: float64


I'm confused with the input unit choosing, year or month. I reference Diebold and Li's paper, and test month.

In [6]:
# Convert time from month to year, b/c we usually discuss annualized rate
year_to_mat = data["Months to Expiry"]/12
month_to_mat = data["Months to Expiry"]
print(month_to_mat)

0       0.933333
1       3.033333
2       7.700000
3      19.900000
4      26.033333
5      44.233333
6      56.400000
7      70.166667
8      82.333333
9      92.933333
10    105.100000
11    120.366667
12    146.233333
13    164.500000
14    228.433333
15    292.333333
16    348.600000
17    470.366667
18    604.300000
Name: Months to Expiry, dtype: float64


??? The yield we download from bloomberg is annualized???
??? The bond price should be qutoed price. So we should build one more Accured Interest function.

In [7]:
# Convert yield in % 
yld = data["Yield"]
print(yld)

0     0.7033
1     0.6612
2     0.7361
3     0.7204
4     0.7254
5     0.7901
6     0.8916
7     0.9584
8     1.0132
9     1.0933
10    1.2032
11    1.3111
12    1.4289
13    1.5624
14    1.8769
15    1.9594
16    2.0078
17    1.9295
18    1.9095
Name: Yield, dtype: float64


## Part 2: Nelson-Siegel Model

### Starting Point: simple regression to find B1, B2 and B3

In [8]:
# Simple regression to estimate beta1,2,3 as start point
lamda = 0.0605

# Prepare x 0,1,2 to do regression
# x_0 = np.ones(len(year_to_mat))

def X_1(x,lamda):
    x_1 = []
    for i in x:
        first = (1-math.exp(-lamda*i))/(lamda*i)
        x_1.append(first)
    return x_1

def X_2(x,lamda):
    x_2 = []
    for i in x:
        second = (1-math.exp(-lamda*i))/(lamda*i) - math.exp(-lamda*i)
        x_2.append(second)
    return x_2

x_1 = X_1(month_to_mat,lamda)
x_2 = X_2(month_to_mat,lamda)


x_reg_sp = pd.DataFrame({'x1':x_1, 'x2':x_2})

In [9]:
def yld_convert(n, yld):
    trans_yld = []
    for i in yld:
        new_y = pow((1 + i),n) - 1
        trans_yld.append(new_y)
    return trans_yld

# Convert annualized yield to monthly yield
y_reg_sp = yld_convert(1/12,yld)
print(y_reg_sp)

[0.04538018100021124, 0.043202198241305556, 0.04704310549384605, 0.04625075923502986, 0.046503815616885325, 0.04971912006587442, 0.054554693108910035, 0.05760894838726549, 0.06004404377652528, 0.06349623346370992, 0.06804075944348065, 0.07230474589767821, 0.07675641929236332, 0.0815682017549817, 0.09205303587528202, 0.09462905730470728, 0.09610984773920217, 0.0937031388956322, 0.09307894794412719]


In [10]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept = True)
 
model.fit(x_reg_sp,y_reg_sp)

print(model.coef_)
print(model.intercept_)

[-0.046069   -0.11301979]
0.09758809566355213


In [11]:
# Payment schedule in years
def pay_sch(t):
    schedule = [t]
    while ((t-6) >= 0):
        t = t-6
        schedule.insert(0,t)
    return schedule


In [12]:
def NS(t, beta):
    y = beta[0] + beta[1] * (1-math.exp(-lamda*t))/(lamda*t) + beta[2] * ((1-math.exp(-lamda*t))/(lamda*t) - math.exp(-lamda*t))
    return y

In [13]:
# Pring all bonds for one day
def pricing_ns(month_to_mat, coupon, beta):
    pv_lst = []
    for i in range(0,len(coupon)):
        schedule = pay_sch(month_to_mat[i])
        pv_principle = 100 / pow(1 + NS(month_to_mat[i], beta)/100, month_to_mat[i])
        pv_coupon = 0
        for t in schedule:
            pv_coupon = pv_coupon + coupon[i] / pow(1+NS(t, beta)/100, t)
        pv = pv_principle + pv_coupon
        pv_lst.append(pv)
    return pv_lst
        

In [ ]:
# Accured Interest

In [14]:
def res(beta, month_to_mat, coupon):
    est_price = pricing_ns(month_to_mat, coupon, beta)
    res = act_price - est_price
    return res

In [15]:
import scipy.optimize as optimize
#import collections

#Param=collections.namedtuple('Param','b0 b1 b2')
#Beta_guess=Param(b0 = 5, b1 = -1.12135598, b2 = -3.01085312)
Beta_guess = np.array([0.09758809566355213,  -0.046069 ,  -0.11301979])
res_1 = optimize.least_squares(res, Beta_guess, args=(month_to_mat, coupon))

In [16]:
print(res_1.x)

print(res_1.optimality)

[ 0.40460247 -0.03767913 -0.37392008]
0.040788656108361465


In [17]:
res_day1 = res(res_1.x, month_to_mat, coupon)
print(res_day1)

0      0.280795
1      0.860876
2     -0.452401
3     -0.314792
4      1.321516
5      7.125224
6      8.015933
7      0.148289
8      5.242884
9      8.759501
10    11.347511
11     8.776837
12   -16.714860
13   -13.640536
14     8.362216
15   -20.321178
16    12.464996
17    13.651077
18     2.195941
Name: Price, dtype: float64
